<a href="https://colab.research.google.com/github/aashmauprety/Machine-Learning/blob/master/PYSYFT_FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install syft

     |████████████████████████████████| 399kB 3.4MB/s 
     |████████████████████████████████| 491kB 24.0MB/s 
     |████████████████████████████████| 81kB 7.9MB/s 
     |████████████████████████████████| 2.1MB 17.4MB/s 
     |████████████████████████████████| 204kB 31.1MB/s 
     |████████████████████████████████| 1.8MB 39.8MB/s 
     |████████████████████████████████| 4.0MB 35.9MB/s 
     |████████████████████████████████| 81kB 4.8MB/s 
     |████████████████████████████████| 450kB 40.8MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 9.0MB 37.0MB/s 
     |████████████████████████████████| 753.4MB 19kB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
     |████████████████████████████████| 2.0MB 43.7MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 61kB 5.4MB/s 
     |████████████████████████████████| 36.9MB 1.3MB/s 
     |█

In [1]:
import torch as th
import syft as sy

In [2]:
hook = sy.TorchHook(th)

In [4]:
bob = sy.VirtualWorker(hook, id = "bob")
alice = sy.VirtualWorker(hook, id = "alice")
john = sy.VirtualWorker(hook, id = "john")

In [5]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [10]:
bob._objects

{87778161136: tensor([1, 2, 3, 4, 5])}

In [8]:
x = x.send(alice)

In [9]:
x

(Wrapper)>[PointerTensor | me:22816854102 -> alice:27038745378]

In [12]:
bob._objects

{87778161136: tensor([1, 2, 3, 4, 5])}

In [13]:
from torch import nn, optim

In [14]:

# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [15]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [16]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)


In [17]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]


In [18]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())


In [19]:
train()

tensor(2.7590, requires_grad=True)
tensor(1.5809, requires_grad=True)
tensor(0.6244, requires_grad=True)
tensor(1.0488, requires_grad=True)
tensor(0.3496, requires_grad=True)
tensor(0.6612, requires_grad=True)
tensor(0.2159, requires_grad=True)
tensor(0.4198, requires_grad=True)
tensor(0.1364, requires_grad=True)
tensor(0.2696, requires_grad=True)
tensor(0.0878, requires_grad=True)
tensor(0.1753, requires_grad=True)
tensor(0.0576, requires_grad=True)
tensor(0.1154, requires_grad=True)
tensor(0.0386, requires_grad=True)
tensor(0.0770, requires_grad=True)
tensor(0.0263, requires_grad=True)
tensor(0.0520, requires_grad=True)
tensor(0.0182, requires_grad=True)
tensor(0.0355, requires_grad=True)
tensor(0.0129, requires_grad=True)
tensor(0.0245, requires_grad=True)
tensor(0.0092, requires_grad=True)
tensor(0.0171, requires_grad=True)
tensor(0.0066, requires_grad=True)
tensor(0.0121, requires_grad=True)
tensor(0.0048, requires_grad=True)
tensor(0.0086, requires_grad=True)
tensor(0.0036, requi